In [1]:
import pandas
import konlpy
import gensim

print(pandas.__version__)
print(konlpy.__version__)
print(gensim.__version__)

1.3.3
0.5.2
4.1.2


In [2]:
import torch
import pandas as pd
from PIL import Image
import glob
import matplotlib.pyplot as plt
import os
import numpy as np

import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [3]:
num_gpus = torch.cuda.device_count()
print("Number of available GPU devices:", num_gpus)

Number of available GPU devices: 1


In [4]:
data = pd.read_csv('~/aiffel/project/FnGuide/finance_data.csv')
data.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [5]:
label_counts = data['labels'].value_counts()

# 요소와 갯수 출력
for label, count in label_counts.items():
    print(f"{label}: {count}")

neutral: 2879
positive: 1363
negative: 604


In [6]:
data = data[['labels', 'kor_sentence']]
data

,labels,kor_sentence
0,neutral,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
...,...,...
4841,negative,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4842,neutral,린쿠스키아의 맥주 판매량은 416만 리터로 6.5% 감소했으며 카우노 알루스의 맥주...
4843,negative,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,negative,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [7]:
data = data[data['labels'] != 'neutral']
data

,labels,kor_sentence
2,negative,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
5,positive,ASPOCOMP의 성장기에 대한 자금 조달은 기술적으로 더 까다로운 HDI 인쇄 회...
6,positive,2010년 4분기 Componenta의 순매출은 전년 동기의 7600만 유로에서 2...
...,...,...
4840,negative,헬싱키 톰슨 파이낸셜 - 카고텍의 주가는 화물 취급 그룹이 4월부터 6월까지 3개월...
4841,negative,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4843,negative,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,negative,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [8]:
data['labels'] = data['labels'].replace({'negative': 0, 'positive': 1})
data

/tmp/ipykernel_396/3214286959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['labels'] = data['labels'].replace({'negative': 0, 'positive': 1})


,labels,kor_sentence
2,0,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
5,1,ASPOCOMP의 성장기에 대한 자금 조달은 기술적으로 더 까다로운 HDI 인쇄 회...
6,1,2010년 4분기 Componenta의 순매출은 전년 동기의 7600만 유로에서 2...
...,...,...
4840,0,헬싱키 톰슨 파이낸셜 - 카고텍의 주가는 화물 취급 그룹이 4월부터 6월까지 3개월...
4841,0,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4843,0,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,0,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [9]:
data[data['kor_sentence'].duplicated()]

,labels,kor_sentence


In [10]:
data = data[~data['kor_sentence'].duplicated()]
data

,labels,kor_sentence
2,0,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
5,1,ASPOCOMP의 성장기에 대한 자금 조달은 기술적으로 더 까다로운 HDI 인쇄 회...
6,1,2010년 4분기 Componenta의 순매출은 전년 동기의 7600만 유로에서 2...
...,...,...
4840,0,헬싱키 톰슨 파이낸셜 - 카고텍의 주가는 화물 취급 그룹이 4월부터 6월까지 3개월...
4841,0,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4843,0,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,0,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [11]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=123)

In [12]:
print('훈련 샘플의 개수 :', len(train_data))
print('테스트 샘플의 개수 :', len(test_data))

훈련 샘플의 개수 : 1573
테스트 샘플의 개수 : 394


In [14]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# RoBERTa 모델과 토크나이저를 로드합니다.
model_name = "roberta-base"

tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [15]:
# 문장과 레이블을 추출합니다.
train_sentences = train_data['kor_sentence'].values
train_labels = train_data['labels'].values

test_sentences = test_data['kor_sentence'].values
test_labels = test_data['labels'].values

In [16]:
# 문장을 토큰화하고 인덱스로 변환합니다.
train_tokens = [tokenizer.encode(sentence, add_special_tokens=True) for sentence in train_sentences]
test_tokens = [tokenizer.encode(sentence, add_special_tokens=True) for sentence in test_sentences]

# 패딩된 텐서로 변환합니다.
train_inputs = torch.nn.utils.rnn.pad_sequence([torch.tensor(tokens) for tokens in train_tokens], batch_first=True)
test_inputs = torch.nn.utils.rnn.pad_sequence([torch.tensor(tokens) for tokens in test_tokens], batch_first=True)

In [17]:
# 레이블을 텐서로 변환합니다.
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

In [18]:
# 데이터를 DataLoader로 만듭니다.
train_dataset = TensorDataset(train_inputs, train_labels)
test_dataset = TensorDataset(test_inputs, test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [19]:
# 모델을 학습시킵니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 10
losses = []  # 평균 손실 값을 저장할 리스트

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)[0]
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    losses.append(avg_loss)  # 평균 손실 값을 리스트에 저장

    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

Epoch 1/10, Average Loss: 0.6278
Epoch 2/10, Average Loss: 0.6211
Epoch 3/10, Average Loss: 0.6149
Epoch 4/10, Average Loss: 0.6177
Epoch 5/10, Average Loss: 0.6110
Epoch 6/10, Average Loss: 0.5759
Epoch 7/10, Average Loss: 0.5306
Epoch 8/10, Average Loss: 0.4876
Epoch 9/10, Average Loss: 0.4666
Epoch 10/10, Average Loss: 0.4383


In [25]:
from sklearn.metrics import classification_report

model.eval()  # 모델을 평가 모드로 설정
predictions_list = []
labels_list = []

with torch.no_grad():  # 그라디언트 계산 비활성화
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)[0]
        _, predictions = torch.max(outputs, dim=1)  # 가장 높은 확률 값을 가진 클래스로 예측
        
        predictions_list.extend(predictions.cpu().numpy())  # 예측값을 리스트에 추가
        labels_list.extend(labels.cpu().numpy())  # 실제 레이블을 리스트에 추가

# 리스트를 NumPy 배열로 변환
predictions_array = np.array(predictions_list)
labels_array = np.array(labels_list)

# 분류 보고서 생성
report = classification_report(labels_array, predictions_array)
print(report)

Test Loss: 0.0000 | Test Accuracy: 0.0000
              precision    recall  f1-score   support

           0       0.81      0.37      0.51       114
           1       0.79      0.96      0.87       280

    accuracy                           0.79       394
   macro avg       0.80      0.67      0.69       394
weighted avg       0.79      0.79      0.76       394

